# ***Grid*** notes (grid.py)

In this notebook will be analyzed the most important functions in the file *grid.py*. This file contains an implementation of the grid on which the cells are placed in the 2D model.

## Classe ***CellList***

# *iter()* e *next()*
```python
def __iter__(self):
    """Needed to iterate on the list object"""
    self._iter_count = -1
    return self

def __next__(self):
    """Needed to iterate on the list object"""
    self._iter_count += 1
    if self._iter_count < self.num_c_cells:
        return self.cancer_cells[self._iter_count]
    elif self._iter_count < self.size:
        return self.healthy_cells[self._iter_count - self.num_c_cells]
    else:
        raise StopIteration
```
* *iter()*:
    * Viene chiamata quando si inizia a iterare su un oggetto di tipo CellList, ad esempio in un ciclo for. In questo caso, imposta un contatore di iterazione *iter_count* a -1, che verrà poi incrementato nella funzione *next()*
    * Dopo aver inizializzato il contatore, la funzione restituisce l'oggetto stesso (*self*). Questo è necessario perché l'oggetto stesso è l'iteratore, ed è un design comune nelle classi personalizzate che implementano l'iterazione.
Per completare l'iterazione, la funzione *iter()* è accompagnata dalla funzione *next()*, che definisce cosa succede ad ogni passo dell'iterazione.
* *next()*
    * Incremento del contatore: La funzione aumenta _iter_count di 1 per passare alla cellula successiva.
    * Restituzione della cellula: Se il contatore è ancora all'interno del numero di cellule cancerose, restituisce la cellula corrispondente dall'elenco cancer_cells. Se è oltre il numero di cellule cancerose ma inferiore al numero totale di celle, restituisce una cellula dall'elenco healthy_cells.
    * Terminazione dell'iterazione: Se il contatore supera il numero totale di cellula, viene sollevata un'eccezione StopIteration, che segnala la fine dell'iterazione.

## Classe ***Grid***

### ***init()***
La funzione ***init()*** della classe Grid in Python è il costruttore che inizializza un'istanza della griglia, definendo vari parametri e strutture dati necessarie per la simulazione

Parametri della funzione:
1. *xsize*: Numero di righe della griglia.
2. *ysize*: Numero di colonne della griglia.
3. *sources*: Numero di fonti di nutrienti nella griglia.
4. *oar*: (opzionale) Descrizione di una zona a rischio (OAR - Organ At Risk) nella griglia.

*   ```python
    # Helpers are useful because diffusion cannot be done efficiently in place.
    # With a helper array of same shape, we can simply compute the result inside the other and alternate between
    # the arrays.
    ```
    * La diffusione del glucosio e dell'ossigeno è un'operazione che richiede di aggiornare i valori di queste sostanze in ogni pixel della griglia in base ai valori dei pixel vicini.
    * "In place" significa che l'operazione modifica direttamente l'array originale. Tuttavia, se provi a fare questo mentre stai ancora leggendo i valori dall'array originale, potresti ottenere risultati errati perché i valori stanno cambiando mentre stai ancora eseguendo il calcolo.
    * Per evitare questi problemi, viene utilizzato un array "helper" (o di supporto) di dimensioni uguali all'array originale. Invece di aggiornare direttamente l'array del glucosio o dell'ossigeno, il calcolo della diffusione viene fatto usando questo array di supporto.
    * Una volta completati i calcoli, i risultati vengono trasferiti all'array principale.

* ```python
    self.cells = np.empty((xsize, ysize), dtype=object)
    for i in range(xsize):
        for j in range(ysize):
            self.cells[i, j] = CellList()
  ```  
  * La classe *CellList* è progettata per gestire una lista di cellule (sane e cancerose) associate a un singolo pixel della griglia. Durante la simulazione, le cellule verranno aggiunte agli oggetti *CellList* associati ai rispettivi pixel della griglia. Ad esempio, se una cellula viene creata o si muove in un pixel (i, j), verrà aggiunta al corrispondente *CellList*, cioè *self.cells[i, j].append(cell)*.

* ```python
    self.sources = random_sources(xsize, ysize, sources)
  ```  
  Genera una lista di coordinate casuali nella griglia in cui saranno presenti le fonti di nutrienti (vasi sanguigni)


* ```python
    # Neigbor counts contain, for each pixel on the grid, the number of cells on neigboring pixels. They are useful
    # as HealthyCells only reproduce in case of low density. As these counts seldom change after a few hundred
    # simulated hours, it is more efficient to store them than simply recompute them for each pixel while cycling.
  ``` 
  * Matrice neigh_counts: È una matrice che ha le stesse dimensioni della griglia e che memorizza il numero di cellule nei pixel vicini per ciascun pixel. Questo evita di dover calcolare ripetutamente questi valori durante la simulazione.
  * Riproduzione delle HealthyCells: Le cellule sane si riproducono solo in ambienti di bassa densità, quindi è essenziale conoscere rapidamente la densità dei pixel circostanti. Il conteggio memorizzato permette di determinare velocemente se le condizioni per la riproduzione sono soddisfatte.
  * Efficienza: Man mano che la simulazione progredisce, la densità cellulare si stabilizza, il che significa che i conteggi dei vicini cambiano meno frequentemente. Questo rende il processo di aggiornamento selettivo dei conteggi più efficiente rispetto al ricalcolo continuo per ogni iterazione della simulazione.

* 
    ```python
    for i in range(xsize):
        self.neigh_counts[i,0] += 3
        self.neigh_counts[i, ysize - 1] += 3
    ```
    * Prima linea nel *for*: Incrementa di 3 il valore in neigh_counts per il pixel della prima colonna (colonna 0). Questo rappresenta il fatto che un pixel sul bordo sinistro ha 3 vicini in meno rispetto ai pixel interni (che ne avrebbero 8). I 3 vicini in meno sono quelli che si troverebbero al di fuori della griglia, a sinistra del pixel.
    * Seconda linea: Lo stesso viene fatto per l'ultima colonna.

    ```python
    for i in range(ysize):
        self.neigh_counts[0, i] += 3
        self.neigh_counts[xsize - 1, i] += 3
    ```
    Come nel caso sopra ma per la prima e l'ultima riga.

    ```python
    self.neigh_counts[0, 0] -= 1
    self.neigh_counts[0, ysize - 1] -= 1
    self.neigh_counts[xsize - 1, 0] -= 1
    self.neigh_counts[xsize - 1, ysize - 1] -= 1
    ```
    Queste linee correggono il conteggio agli estremi della matrice.

    Considerando un valore di 0 per gli elementi interni alla matrice *neigh_counts*, la differenza nel conteggio con il bordo è la seguente

    ```python
    [[5, 3, 3, 3, 5],
    [3, 0, 0, 0, 3],
    [3, 0, 0, 0, 3],
    [3, 0, 0, 0, 3],
    [5, 3, 3, 3, 5]]
    ```

* 
    ```python
    self.oar = oar
    ```
    Assegna semplicemente il valore del parametro *oar* all'attributo dell'istanza *self.oar*. *oar* è un parametro opzionale del costruttore della classe *Grid*. Non è obbligatorio passare un valore per oar quando si crea un'istanza di *Grid*. Se non viene fornito alcun valore, *oar* sarà *None* (il valore predefinito in Python per i parametri non passati). La variabile self.oar nella classe Grid viene utilizzata per rappresentare una zona a rischio (OAR - Organ At Risk) all'interno della griglia. In altre, self.oar viene utilizzata per determinare se una posizione nella griglia è all'interno della zona a rischio e per limitare o modificare il comportamento delle cellule all'interno di tale area. Il metodo *wake_surrounding_oar* utilizza self.oar per determinare quali celle circostanti devono essere "risvegliate" o attivate in base alla loro vicinanza alla zona OAR.

* 
    ```python
    self.center_x = self.xsize // 2
    self.center_y = self.ysize // 2
    ```
    Calcolano le coordinate del centro della griglia lungo gli assi x e y.


### ***count_neigbors() and neighbors()***
*count_neigbors()* calcola i conteggi dei vicini (il numero di celle sui pixel adiacenti) per ogni pixel. Viene ispezionato ogni pixel della matrice e su ciascuno di essi si ha:
```python
self.neigh_counts[i, j] = sum(v for _, _, v in self.neighbors(i, j))
```
dove 
```python
v for _, _, v in self.neighbors(i, j)
```
estrae solo il terzo elemento (*v*) di ciascuna tupla restituita dalla funzione *neighbors(i, j)*. *v* rappresenta il numero di cellule presenti in ogni pixel vicino, compreso il pixel stesso *(i, j)*. Succesivamente si fa la somma di tutte le cellule con *sum()*.

### ***fill_source(), source_move and rand_neigh()***
*fill_source()* itera su tutte le fonti di nutrimento (le fonti, sono coordinate contenute in *self.sources*) e riempie le matrici di glucosio e ossigeno.  C'è una piccola probabilità (1 su 24, poiché *random.randint(0, 23)* genera un numero casuale tra 0 e 23 inclusi) che una fonte si muova. Se il numero casuale è 0, la fonte corrente si sposta in una nuova posizione, determinata dalla funzione *self.source_move*. La funzione determina se la fonte si muove verso il centro del tumore o in una direzione casuale:
* Se il numero casuale generato è minore del numero totale di cellule cancerose (*CancerCell.cell_count*), la fonte si sposta di una posizione verso il centro della griglia (calcolato in base alle coordinate *self.center_x* e *self.center_y*).
* Se il numero casuale è maggiore o uguale a CancerCell.cell_count, la fonte si muove verso un pixel adiacente scelto casualmente tramite la funzione *self.rand_neigh*.

### ***diffuse_glucose(), diffuse_oxygen(), neighbors_glucose(), neighbors_glucose() and neighbors_oxygen()***
Sono responsabili della diffusione di glucosio e ossigeno all'interno della griglia
* *drate* rappresenta la frazione di glucosio (o ossigeno) che si diffonde verso i vicini. Determina quanto velocemente il glucosio o l'ossigeno si diffonde attraverso la griglia. Un valore più alto di drate implica una diffusione più rapida. *(1 - drate)*
* *(1 - drate) x self.glucose*: Questa parte rappresenta la quantità di glucosio (o ossigeno) che rimane nella posizione attuale dopo la diffusione.
* *(0.125 x drate) x self.neighbors_glucose()*: Questa parte rappresenta il contributo del glucosio (o ossigeno) dai vicini. Il fattore 0.125 è derivato dall'assunzione che il contributo di ciascun vicino (8 in totale) sia distribuito 
* *self.neighbors_glucose()* e *self.neighbors_oxygen*: Calcolano il contributo di glucosio e ossigeno dai pixel circostanti a quello in esame. Vengono considerate tutte le direzioni. 
    
    * Esempi di *np.roll()*
        ```python
        >>> x = np.arange(100)
        >>> x = np.reshape(x, (10, 10))
        >>> x
         array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],   
                [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],   
                [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],   
                [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],   
                [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],   
                [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],   
                [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],   
                [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],   
                [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],   
                [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

        >>> np.roll(x, 1, axis=0)   
        array([[90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
                [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
                [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
                [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
                [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
                [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
                [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
                [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
                [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
                [80, 81, 82, 83, 84, 85, 86, 87, 88, 89]])
        >>> np.roll(x, 1, axis=1) 
         array([[ 9,  0,  1,  2,  3,  4,  5,  6,  7,  8],
                [19, 10, 11, 12, 13, 14, 15, 16, 17, 18],
                [29, 20, 21, 22, 23, 24, 25, 26, 27, 28],
                [39, 30, 31, 32, 33, 34, 35, 36, 37, 38],
                [49, 40, 41, 42, 43, 44, 45, 46, 47, 48],   
                [59, 50, 51, 52, 53, 54, 55, 56, 57, 58],   
                [69, 60, 61, 62, 63, 64, 65, 66, 67, 68],   
                [79, 70, 71, 72, 73, 74, 75, 76, 77, 78],   
                [89, 80, 81, 82, 83, 84, 85, 86, 87, 88],   
                [99, 90, 91, 92, 93, 94, 95, 96, 97, 98]]) 
        ```

    * Gestione dei Bordi: Dopo aver effettuato gli spostamenti, è necessario assicurarsi che gli elementi che si trovano ai bordi della griglia non ricevano contributi da posizioni al di fuori della griglia quindi si pongono uguali a zero (non ricevono nessuna quantità di ossigeno e glucosio)

    *  Somma dei Contributi: Alla fine, tutti i contributi dai vicini sono sommati per ottenere il totale del glucosio (o ossigeno) che diffonde in una cella specifica.


### ***cycle_cells(), rand_min(), rand_neigh(), find_hole(), add_neigh_count()***

**Funzionamento generale di cycle_cells()** 

Il ciclo cellulare implementato nella funzione cycle_cells() della classe Grid gestisce il comportamento delle cellule presenti nella griglia durante ogni iterazione della simulazione. Il ciclo cellulare delle cellule include processi come il consumo di nutrienti, la divisione (mitosi), e la morte cellulare. La funzione *cycle_cells()* invoca il metodo *cycle()* su ogni cellula per simulare il loro ciclo vitale. Quando la funzione cycle() di una cellula viene chiamata, il suo comportamento viene determinato in base al tipo di cellula, alle condizioni locali (come la quantità di glucosio, ossigeno e la densità cellulare nei pixel vicini), e al suo stadio di sviluppo. Ecco i passaggi generali del ciclo cellulare:

1. **Consumo di Nutrienti**
    * Consumo di Nutrienti: Ogni cellula consuma glucosio e ossigeno dal pixel in cui si trova. La quantità esatta di nutrienti consumati dipende dal tipo di cellula e dal suo stadio di crescita.

    * I livelli di glucosio e ossigeno del pixel vengono decrementati in base al consumo della cellula:
        ```python
        self.glucose[i, j] -= res[0]
        self.oxygen[i, j] -= res[1]
        ```

2. **Decisione di Mitosi**
    * Durante il ciclo cellulare, la cellula può decidere di dividersi, cioè di eseguire la mitosi. Questo dipende da diversi fattori:
        * Cellule Sane: Possono dividersi se la densità delle cellule nei pixel vicini è bassa e se ci sono abbastanza nutrienti disponibili.
        * Cellule Cancerose: Tendono a dividersi più aggressivamente e possono farlo anche in condizioni meno favorevoli.
        * Cellule OAR: Possono risvegliarsi o dividersi in risposta alla presenza di cellule tumorali vicine.
    * Se la cellula decide di dividersi, la funzione *cycle()* restituisce un terzo valore (*res[2]*) che indica il tipo di mitosi da eseguire.

3. **Posizionamento della Nuova Cellula**

    * Quando una cellula si divide, la nuova cellula viene posizionata in un pixel vicino:
        * Per le cellule sane, viene cercata una posizione a bassa densità tramite la funzione *rand_min()*.
        * Per le cellule cancerose, viene selezionato un pixel vicino casualmente tramite la funzione *rand_neigh*().
    * La nuova cellula viene aggiunta alla lista delle cellule nel pixel appropriato e il conteggio delle cellule nei pixel vicini viene aggiornato.
4. **Morte Cellulare**
    * Le cellule che non sopravvivono (ad esempio, a causa della mancanza di nutrienti) vengono rimosse dalla griglia.
    * Dopo ogni iterazione del ciclo, la funzione *delete_dead()* viene chiamata per rimuovere le cellule morte:
    * Se delle cellule vengono rimosse, il conteggio delle cellule nei pixel vicini viene decrementato.

***Nota***: 
* Per una cellula sana: Se la cellula non si divide, può progredire verso un altro stadio di sviluppo, oppure potrebbe restare nello stesso stadio o morire se i nutrienti sono insufficienti.
* Per una cellula cancerosa:  Le cellule cancerose possono continuare a dividersi finché non vengono esaurite le risorse locali o finché non vengono distrutte da interventi esterni (radioterapia).

**Passaggi Principali della Funzione**

1. **Inizializzazione**

    ```python
    to_add = []
    tot_count = 0
    ```
    * *to_add*: Una lista che raccoglie le nuove cellule da aggiungere alla griglia dopo la mitosi.  Durante l'iterazione attraverso ogni cellula nella griglia, può verificarsi la divisione cellulare. Quando ciò accade, una nuova cellula deve essere creata e posizionata in una posizione adiacente. Anziché aggiungere immediatamente le nuove cellule alla griglia, che potrebbe interferire con l'attuale processo di iterazione, le nuove cellule vengono memorizzate nella lista to_add. Questa lista contiene tuple che specificano la posizione (x, y) in cui la nuova cellula deve essere posizionata e la cellula stessa.
    * *tot_count*:  Tiene traccia del numero totale di cicli cellulari eseguiti durante un'iterazione su tutte le celle della griglia.

2. **Iterazione sulla Griglia**

    ```python
    for i in range(self.xsize):
        for j in range(self.ysize):  # For every pixel
            for cell in self.cells[i, j]:
                ...
    ```
    * Viene iterato su ogni pixel della griglia (ogni coppia di coordinate *(i, j)*).
    * Per ogni pixel, la funzione itera su tutte le cellule presenti in quel pixel.

3. **Ciclo della Cellula**

    ```python
    res = cell.cycle(self.glucose[i, j],  self.neigh_counts[i, j], self.oxygen[i, j])
    ```
    * Per ogni cellula, viene invocato il metodo cycle() della cellula stessa, che gestisce le operazioni legate al ciclo vitale della cellula, come il consumo di nutrienti e la valutazione della divisione cellulare.
    * *res* contiene i risultati del ciclo della cellula, i quali includono il consumo di glucosio e ossigeno, e un eventuale segnale di mitosi.

4. **Gestione della Mitosi**
    ```python
    if len(res) > 2:
        if res[2] == 0:  # Mitosis of a healthy cell
         ...
        elif res[2] == 1:  # Mitosis of a cancer cell
            ...
        elif res[2] == 2:  # Wake up surrounding oar cells
            ...
        elif res[2] == 3:
            ...
    ```
    * Se res contiene più di due elementi, significa che la cellula ha deciso di dividersi o di svolgere un'altra azione particolare.
    * A seconda del valore di *res[2]*, la funzione gestisce diversi tipi di azioni:
        * *0*: Mitosi di una cellula sana, con creazione di una nuova cellula in una posizione a bassa densità vicina.
        * *1*: Mitosi di una cellula cancerosa, con creazione di una nuova cellula in una posizione adiacente casuale.
        * *2*: Attivazione delle cellule OAR circostanti.
        * *3*: Ricerca di un buco dove posizionare una nuova cellula OAR.

5. **Aggiornamento dei Nutrienti**
    ```python
    self.glucose[i, j] -= res[0]
    self.oxygen[i, j] -= res[1]
    ```
    * I livelli di glucosio e ossigeno nel pixel corrente vengono ridotti in base al consumo della cellula.

6. **Rimozione delle Cellule Morte**
    ```python
    count = len(self.cells[i, j])
    self.cells[i, j].delete_dead()
    if len(self.cells[i, j]) < count:
        self.add_neigh_count(i, j, len(self.cells[i, j]) - count)
    ```
    * Le cellule morte vengono rimosse dal pixel corrente attraverso la chiamata a *delete_dead()*.
    * Se il numero di cellule nel pixel è cambiato, viene aggiornato il conteggio dei vicini.

7. **Aggiunta delle Nuove Cellule**
    ```python
    for i, j, cell in to_add:
        self.cells[i, j].append(cell)
        self.add_neigh_count(i, j, 1)
    ```
    * Le nuove cellule generate dalla mitosi vengono aggiunte ai pixel appropriati.
    * Anche il conteggio dei vicini viene aggiornato per riflettere l'aggiunta delle nuove cellule.

8. **Restituzione del Conteggio Totale**
    ```python
    return tot_count
    ```
    * La funzione restituisce il numero totale di cicli cellulari effettuati, che può essere utilizzato per monitorare lo stato della simulazione.
        
    
**Funzioni utilizzate all'interno di *cycle_cells()***:
* *rand_min()*: La funzione ha il compito di identificare e restituire un pixel vicino con la densità di celle più bassa. Questa funzione viene utilizzata principalmente per decidere in quale direzione una nuova cellula (ad esempio, una cellula che si sta dividendo) dovrebbe spostarsi o essere creata, cercando un'area meno affollata.
    * *max*: Il massimo numero di cellule consentito nel pixel di destinazione. La funzione restituirà None se tutti i pixel vicini hanno una densità maggiore o uguale a questo valore.
    * 
        ```python
        v = 1000000
        ind = []
        ```
        *v* viene inizializzato a un valore molto grande (1000000), che rappresenta la densità minima iniziale da cercare. *ind* è una lista vuota che verrà utilizzata per memorizzare le coordinate dei pixel vicini con la densità di celle minima.
    * La funzione restituisce *None* se tutti i vicini hanno una densità di celle pari o superiore a *max*. Altrimenti restituisce una tupla contenente le coordinate *(x, y)* della posizione trovata.
* *HealthyCell(4)*: Oggetto (un'istanza) della classe *HealthyCell*. In questo caso , la cellula sarà inizialmente nello stadio quiescente (stadio 4). Lo stesso vale per *CancerCell(0)*. Attributi dell'oggetto: 
    * *a.stage = 4*: indica che la cellula è nello stadio quiescente.
    * *a.age = 0*: l'età della cellula è inizialmente impostata a 0.
    * *a.alive = True*: la cellula è inizialmente viva.
    * *a.efficiency*: l'efficienza della cellula è impostata in base a un fattore casuale che modula l'assorbimento medio di glucosio.
    * *a.oxy_efficiency*: l'efficienza di consumo di ossigeno è anch'essa basata su un fattore casuale.
    * *a.oxy_efficiency*: l'efficienza di consumo di ossigeno è anch'essa basata su un fattore casuale.

    Per quanto riguarda *OARCell(0, 5)*, il primo parametro rappresenta sempre lo stadio iniziale della cellula mentre il secondo rappresenta il "worth" (valore) della cellula, che viene utilizzato per determinare l'importanza o il tipo della cellula rispetto ad altre cellule nello stesso ambiente.

* *rand_neigh()*: La funzione controlla tutti i pixel vicini e verifica se il pixel in considerazione rispetti la seguente condizione: (§)
    ```python
    i+j <= self.oar[0] + self.oar[1]
    ```

    Successivamente si ha la linea
    ```python
    [c for c in self.cells[i, j] if isinstance(c, OARCell)]
    ```
    che crea una nuova lista che contiene solo le cellule di tipo OARCell presenti nel pixel (i, j). A ciascuna cellula viene impostato lo stage e l'età a 0.

* *find_hole()*:  Ha lo scopo di identificare un "buco" nei dintorni di una cella specifica, ovvero una posizione nella griglia che è relativamente vuota o priva di cellule OAR (Organ At Risk).

    * 
        ```python
        if (i >= 0 and i < self.xsize and j >= 0 and j < self.ysize and i + j <= self.oar[0] + self.oar[1] ):
        ```
        Per ogni posizione (i, j), verifica che sia all'interno dei limiti della griglia. Inoltre, verifica che la somma degli indici (i + j) sia inferiore o uguale alla somma degli indici del centro della zona OAR (self.oar[0] + self.oar[1]), il che implica che la posizione si trova entro una distanza valida dalla zona OAR (§) (ç).

    * 
        ```python
        if len([c for c in self.cells[i, j] if isinstance(c, OARCell)]) == 0:
        l.append((i, j, len(self.cells[i, j])))
        ```
        Se la posizione *(i, j)* non contiene cellule OAR, viene aggiunta alla lista l insieme al numero di cellule totali presenti in quella posizione. 
        
        * Se la lista *l* è vuota (cioè non sono stati trovati buchi), la funzione restituisce *None*.
        * Se invece sono stati trovati buchi, la funzione seleziona quello con il minor numero di cellule (valore minimum) e restituisce la posizione corrispondente.
* *add_neigh_count()*:  Aggiorna il conteggio dei vicini di un pixel specifico nella griglia.

### ***irradiate()***
Questa funzione applica una dose di radiazioni alle cellule, calcolando gli effetti in base alla distanza dal centro della radiazione e alla concentrazione di ossigeno locale, che influisce sull'efficacia della radioterapia.

Parametri della funzione: 
* *dose*: La quantità di radiazione da applicare.
* *center*: La posizione (x, y) sulla griglia da cui parte l'irradiazione. Se non specificato, viene calcolato automaticamente come il centro del tumore.
* *rad*: Il raggio dell'area da irradiare. Se non specificato, viene calcolato in base alla dimensione del tumore.

1. Determinazione del Centro e del Raggio della Radiazione:
    ```python
    if center is None:
        self.compute_center()
        x, y = self.center_x, self.center_y
    else:
        x, y = center
    radius = self.tumor_radius(x, y) if rad == -1 else rad
    if radius == 0:
        return

    ```

    * Se il parametro center non è specificato, la funzione *compute_center()* calcola il centro del tumore, e il raggio viene determinato con *tumor_radius()*.
    * Per quanto riguarda il raggio:
        * Se *rad* è un parametro della funzione *irradiate()* si usa quello. Se, al contrario, il raggio non è un parametro della funzione, *rad* sarà uguale a *-1* e quindi verrà calcolato con la funzione *tumor_radius()*.
        * Successivamente si controlla se il raggio *radius* sia uguale a 0. In questo caso la funzione *irradiate()* termina immediatamente, poiché non c'è nulla da irradiare.
    
2. Calcolo del Moltiplicatore della Dose:

    ```python
    multiplicator = get_multiplicator(dose, radius)
    ```
    * La funzione *get_multiplicator()* calcola un fattore di moltiplicazione per la dose in base al raggio e alla dose totale. Serve per modulare l'intensità della radiazione applicata alle cellule in funzione della distanza dal centro della radiazione. In un contesto reale di radioterapia, la dose di radiazione non è distribuita uniformemente su tutta l'area trattata; tende invece a decrescere con la distanza dal punto centrale di irradiazione. Il fattore di moltiplicazione per la dose tiene conto di questa diminuzione, permettendo di simulare come la radiazione colpisce più intensamente le cellule vicine al centro dell'irradiazione rispetto a quelle più lontane.

3. Parametri per l'Oxygen Enhancement Ratio (OER):
    ```python
    oer_m = 3.0
    k_m = 3.0
    ```
    * Questi parametri (*oer_m* e *k_m*) sono utilizzati per calcolare l'effetto dell'ossigeno sull'efficacia della radiazione. Vedi punto successivo.

4. Applicazione della Radiazione:

    ```python
    for i in range(self.xsize):
        for j in range(self.ysize):
            dist = math.sqrt((x-i)**2 + (y-j)**2)
            if dist < 3*radius:
                omf = (self.oxygen[i, j] / 100.0 * oer_m + k_m) / (self.oxygen[i,j] / 100.0 + k_m) / oer_m
                for cell in self.cells[i, j]:
                    cell.radiate(scale(radius, dist, multiplicator) * omf)
                count = len(self.cells[i, j])
                self.cells[i, j].delete_dead()
                if len(self.cells[i, j]) < count:
                    self.add_neigh_count(i, j, len(self.cells[i, j]) - count)
    ```
    * La funzione scorre ogni pixel della griglia e calcola la distanza dal centro della radiazione.
    * Se la distanza è inferiore a tre volte il raggio, si applica la radiazione.
    * Oxygen Modification Factor (OMF): L'efficacia della radiazione è modificata dalla quantità di ossigeno disponibile nel pixel. $$\text{OMF} = \frac{\left(\frac{\text{O}_2[i, j]}{100} \times \text{OER}_{\text{m}} + k_{\text{m}}\right)}{\left(\frac{\text{O}_2[i, j]}{100} + k_{\text{m}}\right)} \times \frac{1}{\text{OER}_{\text{m}}}$$
        * *self.oxygen[i, j]*: Rappresenta la quantità di ossigeno disponibile nel pixel della griglia che si sta analizzando. Questa quantità è normalizzata dividendola per 100.0 per adattarla alla scala del modello.
        * *oer_m* (Oxygen Enhancement Ratio massimo): Rappresenta il fattore di miglioramento dovuto all'ossigeno. oer_m indica il massimo potenziale di aumento della sensibilità alla radiazione in presenza di ossigeno.
        * *k_m*: Costante di saturazione che regola il contributo relativo dell'ossigeno all'effetto radioterapico. Essa serve a modellare la risposta non lineare dell'effetto dell'ossigeno in funzione della sua concentrazione.
    * Spiegazione della formula:
        * Numeratore: $$\left(\frac{\text{O}_2[i, j]}{100} \times \text{OER}_{\text{m}} + k_{\text{m}}\right)$$
        Esprime la combinazione della quantità di ossigeno disponibile (normalizzata) e del contributo di saturazione rappresentato da $k_m$. Il fattore $k_m$ è introdotto per modellare la saturazione, ovvero il punto in cui aumentare la concentrazione di ossigeno ha un effetto ridotto sull'aumento della radiosensibilità delle cellule. Quando la concentrazione di ossigeno è bassa, il termine $\frac{\text{O}_2[i, j]}{100}$ è piccolo e il valore complessivo della somma $\left(\frac{\text{O}_2[i, j]}{100} + k_{\text{m}}\right)$ è dominato da $k_m$. Questo riflette la condizione in cui la sensibilità delle cellule è meno dipendente dall'ossigeno. Quando la concentrazione di ossigeno aumenta, $\frac{\text{O}_2[i, j]}{100}$ diventa significativo rispetto a $k_m$, e quindi l'effetto dell'ossigeno diventa più rilevante. **Effetto base**: $k_m$ rappresenta una sorta di effetto di base o una soglia sotto la quale l'aumento dell'ossigeno non ha un impatto così drastico (quando l'ossigeno è basso, anche se lo aumeto non cambia nulla poichè domina comunque $k_m$). Questo stabilisce un limite inferiore per la radiosensibilità, evitando che il fattore *omf* diventi troppo piccolo in condizioni di ipossia estrema.
        * Denominatore: $$\left(\frac{\text{O}_2[i, j]}{100} + k_{\text{m}}\right)$$
        Simile al numeratore, ma senza il fattore oer_m. Questo passaggio normalizza il risultato per assicurare che il fattore omf si mantenga in un range che modula l'efficacia della radiazione sulla base della quantità di ossigeno.
        * Divisione finale per *oer_m*: $$\frac{1}{\text{OER}_{\text{m}}}$$
        La divisione finale per oer_m riporta il valore del omf a una scala relativa, assicurando che l'effetto dell'ossigeno sia espresso come un fattore di modifica rispetto al massimo potenziale di miglioramento dato dall'ossigeno.

    * Scala della Radiazione: La funzione *scale()* calcola l'intensità della radiazione in base alla distanza dal centro e al moltiplicatore della dose.
    * Irradiazione delle Cellule: Ogni cellula nel pixel viene irradiata chiamando il metodo *radiate()* della cellula, che gestisce gli effetti della radiazione su di essa.
    * Eliminazione delle Cellule Morte: Dopo l'irradiazione, le cellule morte vengono rimosse e il conteggio dei vicini viene aggiornato se necessario.

5. Restituzione del Raggio
    La funzione restituisce il raggio dell'area irradiata.

### ***compute_center()***
La funzione compute_center() ha lo scopo di calcolare il centro del tumore, cioè il punto medio delle posizioni delle cellule cancerose nella griglia.

Per ogni pixel della griglia che contiene cellule cancerose, la posizione di quel pixel contribuisce alla determinazione del centro del tumore in base al numero di cellule cancerose presenti in quel pixel. In altre parole, più cellule ci sono in un certo pixel, maggiore sarà il peso di quel pixel nel calcolo del centro. Le formule per il calcolo del centro sono le seguenti:
$$\text{sum\_x} = \sum_{(i,j) \in \text{griglia}} i \times \text{ccell\_count}$$
$$\text{sum\_y} = \sum_{(i,j) \in \text{griglia}} j \times \text{ccell\_count}$$
$$\text{center\_x} = \frac{\text{sum\_x}}{\text{count}}$$
$$\text{center\_y} = \frac{\text{sum\_y}}{\text{count}}$$
$$\text{count} = \sum_{(i,j) \in \text{griglia}} \text{ccell\_count}$$
Dove *sum_x* e *sum_y* sono rispettivamente la somma ponderata delle coordinate x e y di tutte le cellule cancerose nella griglia, *count* il numero totale di cellule cancerose e *center_x*, *center_y* le coordinate del centro del tumore.

### ***tumor_radius()***
Calcola il raggio del tumore a partire dal centro del tumore stesso. La condizione che troviamo nel codice della funzione
```python
self.cells[i, j][ 0].__class__ == CancerCell:
```
è necessaria poichè ogni pixel della griglia (*self.cells[i, j]*) può contenere più cellule, rappresentate nella lista CellList. Tuttavia, per verificare se c'è una cellula cancerosa in quel pixel, non è necessario esaminare tutte le cellule nella lista. Controllare solo la prima cellula della lista (*self.cells[i, j][0]*) è un'ottimizzazione, assumendo che le cellule siano inserite nella lista con un certo ordine





# Info aggiuntive

## Info sulla funzione erf()
**Come mai la diffusione della radiaazione nei tessuti viene trattata come una Gaussiana?** La radiazione applicata al tessuto biologico non si distribuisce uniformemente, ma tende a diffondersi attorno al punto di applicazione. La distribuzione gaussiana è utilizzata in questo contesto perchè descrivere bene come l'intensità della radiazione decresca man mano che ci si allontana dal centro del fascio.
# erf()
La funzione *erf()* calcola quella che è conosciuta come la **funzione di errore gaussiana**, che è strettamente correlata all'integrale della distribuzione normale (o distribuzione gaussiana). Più precisamente, la funzione di errore è utilizzata per calcolare la probabilità cumulativa associata alla distribuzione normale standardizzata, ovvero il livello di accuratezza o incertezza nell'integrazione di una funzione gaussiana tra 0 e un valore *x*. E' definita come:
$$\text{erf}(x) = \frac{2}{\sqrt{\pi}} \int_{0}^{x} e^{-t^2} \, dt$$
La funzione di errore è strettamente correlata alla distribuzione normale (o gaussiana). Nelle applicazioni di probabilità e statistica, viene utilizzata per calcolare le probabilità cumulative in una distribuzione normale standardizzata.

# erf() e CDF
La Gaussiana ha la seguente forma:

$$f(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{(x - \mu)^2}{2 \sigma^2}}$$
Dove:

* $\mu$ è la media della distribuzione (centro della curva).
* $\sigma$ è la deviazione standard della distribuzione (larghezza della curva).
* $\sigma^2$ è la varianza.

Quando si "integra" una funzione gaussiana, si sta calcolando l'area sotto la curva tra due valori *a* e *b*. Questo è importante in molte applicazioni, ad esempio quando si calcola la probabilità che una variabile aleatoria assuma un valore compreso tra *a* e *b*. Infati se noi integriamo da $-\infty$ a $+\infty$ otteniamo 1 il che rappresenza la probabilità totale.
Introduciamo la CDF e la erf():
* **CDF**: Quando si integra una gaussiana su un intervallo limitato, ad esempio da $-\infty$ a *x*, si calcola la funzione di distribuzione cumulativa (CDF). Questo rappresenta la probabilità che una variabile aleatoria segua la distribuzione normale e sia minore o uguale a *x*:
$$\Phi(x) = \int_{-\infty}^{x} \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{(t - \mu)^2}{2 \sigma^2}} \, dt$$
* La funzione di errore gaussiana (*erf()*) è definita coem segue:
$$\text{erf}(x) = \frac{2}{\sqrt{\pi}} \int_{0}^{x} e^{-t^2} \, dt$$
Questo è strettamente correlato all'integrazione della gaussiana, e viene utilizzato per calcolare probabilità cumulative.

**Legame tra erf() e $\Phi$(x):**
La funzione di errore (erf) viene utilizzata per semplificare il calcolo della CDF della distribuzione normale standard.
$$\Phi(x) = \frac{1}{2} \left( 1 + \text{erf} \left( \frac{x}{\sqrt{2}} \right) \right)$$
La funzione di errore erf è strettamente correlata all'integrale della distribuzione normale standard perché entrambe calcolano aree sotto una curva gaussiana. Tuttavia, la *erf* viene definita in termini di un integrale esponenziale più semplice e viene utilizzata per calcolare in modo efficiente la probabilità cumulativa.

# Differenza tra due *erf()*
**Cosa vuol dire fare la differenza tra due CDF?** 
Fare la differenza tra due funzioni di distribuzione cumulativa (CDF) significa calcolare la probabilità che una variabile aleatoria con una determinata distribuzione cada all'interno di un intervallo specifico. Questo concetto è particolarmente utile per determinare la probabilità che una variabile aleatoria assuma valori compresi tra due limiti *a* e *b*. Matematicamente:
Data una CDF
$$\Phi(x) = P(X \leq x)$$
si ha che dati due valori $a$ e $b$ coin $a<b$, la differenza tra le CDF valutate in questi punti.

La CDF $\Phi(x)$ di una variabile aleatoria $x$ rappresenta la probabilità che $X$ assuma un valore minore o uguale a $x$:

$$\Phi(x) = P(X \leq x)$$

Se si hanno due valori $a$ e $b$ con $a<b$,  la differenza tra le CDF valutate in questi punti, $\Phi(b) - \Phi(a)$, rappresenta la probabilità che la variabile aleatoria $X$ assuma un valore compreso tra $a$ e $b$:
$$\Phi(b) - \Phi(a) = P(a < X \leq b)$$

**Cosa vuol dire fare la differenza tra due erf()?**
La differenza tra due valori della funzione di errore $\text{erf(b)}$ e $\text{erf(a)}$ rappresenta una quantità legata alla probabilità cumulativa che una variabile casuale, seguendo una distribuzione normale standardizzata, si trovi in un determinato intervallo tra $a$ e $b$
$$\text{erf}(b) - \text{erf}(a)$$
Anche se erf(z) non è direttamente la CDF della distribuzione normale, è legata alla probabilità cumulativa (è proporzionale alla CDF)

# Come mai si preferisce usare erf() invece della CDF?
La funzione erf(z) è direttamente collegata all'integrale della distribuzione normale standardizzata (senza trasformazioni ulteriori). Poiché la distribuzione normale standardizzata ha un'espressione complessa a causa dell'esponente quadratico nella sua funzione di densità di probabilità (PDF), l'integrale di questa espressione è difficile da risolvere esattamente. La funzione di errore fornisce una soluzione semplificata a questo integrale.

Quando si lavora con funzioni che coinvolgono distribuzioni gaussiane o normali, la funzione erf è più veloce da calcolare in molte implementazioni numeriche rispetto alla CDF della normale standard. Questo è particolarmente vero perché la erf ha implementazioni ben ottimizzate nelle librerie di calcolo numerico (come in Python o in altre librerie matematiche), che gestiscono l'integrazione della funzione gaussiana in modo efficiente.

Esistono altri vantaggi per cui si preferisce la funzione *erf()*

# Info sul calcolo della distribuzione della radiazione


## conv(14,0)
Come detto precedentemente, *conv()* produce un valore che è strettamente legato alla proprietà cumulativa. *conv(14,0)* ci dice come si distribuisce la dose nel centro del tumore ovvero: Quanto "intensamente" è concentrata la radiazione in base alla funzione di distribuzione che stiamo considerando che nel nostro caso è la Gaussiana (il profilo di distribuzione della dose attorno al punto di impatto, cioè nel centro del tumore, è lo stesso profilo di una Gaussiana).

conv(14,x X 10/radius) calcola come si distribuisce la radiazione fuori dal centro dove (x*10/radius) è la distanza dal centro del tumore normalizzata. Questa non è una misura di radaizione ma di distribuzione. Dobbiamo quindi moltiplicarlo per la radiazione. 

## Moltiplicatere
**Come mai usiamo il moltiplicatore?**
In un modello di simulazione come quello descritto, la dose di radiazione non viene applicata in modo uniforme su tutta la griglia; invece, viene distribuita secondo una certa funzione che dipende dalla distanza dal centro del tumore. Tipicamente, la dose è maggiore vicino al centro del tumore e diminuisce man mano che ci si allontana. A causa di questa distribuzione non uniforme, se somministrassimo semplicemente la dose di radiazione senza alcuna normalizzazione, la dose totale effettiva applicata potrebbe non corrispondere alla dose desiderata. Potrebbe essere troppo alta o troppo bassa.

Il **moltiplicatore** è definito come:
$$\text{Multiplicator} = \frac{\text{Dose}}{\text{Distribuzione Naturale (conv())}}$$

**Esempio di Distribuzione della Radiazione: Senza e Con Moltiplicatore**

In questo esempio, mostriamo sia la distribuzione della radiazione **senza l'uso del moltiplicatore**, sia la distribuzione **corretta** con l'uso del moltiplicatore. Entrambe le tabelle mostrano i risultati in unità di **Gray (Gy)**.

Supponiamo di voler distribuire una **dose totale desiderata di 200 Gray (Gy)** su una griglia che copre un'area attorno al tumore.

**Passo 1: Distribuzione Naturale della Radiazione**

Immaginiamo che la funzione *conv(14, x)* fornisca la seguente distribuzione naturale della radiazione (valori ipotetici) in 5 punti attorno al centro del tumore:

| Punto | Distanza dal Centro \(x\) | Distribuzione Naturale (conv) |
|-------|----------------------------|------------------------------|
| 1     | 0                          | 40                           |
| 2     | 1                          | 25                           |
| 3     | 2                          | 15                           |
| 4     | 3                          | 10                           |
| 5     | 4                          | 10                           |

**Somma della distribuzione naturale**:

$$\text{Somma naturale} = 40 + 25 + 15 + 10 + 10 = 100 \, \text{unità}$$

Quindi, senza applicare alcuna correzione, la somma totale della distribuzione naturale sarebbe **100 unità** di radiazione, mentre vogliamo distribuire **200 Gray (Gy)**.

**Passo 2: Distribuzione della Radiazione Senza Moltiplicatore**

Se non applicassimo alcuna correzione (cioè non usassimo il moltiplicatore), i valori della radiazione distribuita su ogni punto sarebbero esattamente quelli della distribuzione naturale:

| Punto | Distanza dal Centro \(x\) | Distribuzione Naturale (unità) | Dose di Radiazione (Gy) |
|-------|----------------------------|-------------------------------|-------------------------|
| 1     | 0                          | 40                            | 40 Gy                   |
| 2     | 1                          | 25                            | 25 Gy                   |
| 3     | 2                          | 15                            | 15 Gy                   |
| 4     | 3                          | 10                            | 10 Gy                   |
| 5     | 4                          | 10                            | 10 Gy                   |

**Somma della dose totale senza moltiplicatore**:

$$\text{Somma totale senza correzione} = 40 \, \text{Gy} + 25 \, \text{Gy} + 15 \, \text{Gy} + 10 \, \text{Gy} + 10 \, \text{Gy} = 100 \, \text{Gy}$$

**Osservazione**:

Senza applicare il moltiplicatore, la somma totale della radiazione distribuita è **100 Gy**, che è **la metà** della dose totale desiderata (200 Gy). Questo significa che la distribuzione naturale **non è sufficiente** a raggiungere la dose totale desiderata.

**Passo 3: Calcolo del Moltiplicatore**

Per correggere questa distribuzione e distribuire la dose totale desiderata di **200 Gray (Gy)**, calcoliamo il **moltiplicatore**:

$$\text{Moltiplicatore} = \frac{\text{Dose totale desiderata (Gy)}}{\text{Somma naturale}} = \frac{200 \, \text{Gy}}{100} = 2$$
**Passo 4: Distribuzione della Radiazione Correttamente Moltiplicata**

Ora applichiamo il moltiplicatore di **2** a ogni valore della distribuzione naturale per ottenere la dose corretta in Gray:

| Punto | Distanza dal Centro \(x\) | Distribuzione Naturale (unità) | Dose di Radiazione (Gy) Senza Correzione | Dose di Radiazione Correttamente Moltiplicata (Gy) |
|-------|----------------------------|-------------------------------|-----------------------------------------|---------------------------------------------------|
| 1     | 0                          | 40                            | 40 Gy                                   | 40 × 2 = 80 Gy                                    |
| 2     | 1                          | 25                            | 25 Gy                                   | 25 × 2 = 50 Gy                                    |
| 3     | 2                          | 15                            | 15 Gy                                   | 15 × 2 = 30 Gy                                    |
| 4     | 3                          | 10                            | 10 Gy                                   | 10 × 2 = 20 Gy                                    |
| 5     | 4                          | 10                            | 10 Gy                                   | 10 × 2 = 20 Gy                                    |

**Somma Totale della Dose Correttamente Moltiplicata**

La somma delle dosi corrette è:

$$\text{Somma totale corretta} = 80 \, \text{Gy} + 50 \, \text{Gy} + 30 \, \text{Gy} + 20 \, \text{Gy} + 20 \, \text{Gy} = 200 \, \text{Gy}$$

**Osservazione**

Dopo aver applicato il moltiplicatore, la somma totale della dose distribuita è esattamente **200 Gy**, come desiderato. Ogni punto ha ricevuto una dose di radiazione **corretta** in modo che la somma totale delle dosi rispetti il vincolo della dose totale desiderata.

## scale()
La funzione *scale()* fornisce la dose effettiva della radiazione.



## Da vedere
Modello cellulare di Michaelis-Menten Modificato e altri modelli.

Vedi le formule usate per i moltiplicatori.